In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, 
                             recall_score, f1_score, confusion_matrix)
import joblib

# Load and preprocess data
df = pd.read_csv("ddos_dataset.csv")
df = df.drop(['timestamp', 'source_ip', 'destination_ip', 'source_port'], axis=1)
df['tcp_flags'] = df['tcp_flags'].replace('', 'None')
df['destination_port'] = df['destination_port'].astype(str)

X = df.drop('label', axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['packet_count', 'packet_size', 'duration']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['protocol', 'destination_port', 'tcp_flags'])
])

# Build and train model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        class_weight='balanced',
        random_state=42
    ))
])

model.fit(X_train, y_train)

# Evaluate performance
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save model
joblib.dump(model, 'ddos_detector.pkl')

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
Confusion Matrix:
[[100   0]
 [  0 100]]


['ddos_detector.pkl']

In [3]:
import pandas as pd
import joblib

# Load the trained model
model = joblib.load('ddos_detector.pkl')

# Input features format
input_data = {
    'destination_port': '443',       # As string
    'protocol': 'TCP',              # ['TCP', 'UDP', 'ICMP']
    'packet_count': 95,           # Numeric value
    'packet_size': 273,            # Numeric value
    'duration': 53,                  # Numeric value (seconds)
    'tcp_flags': 'ACK'             # ['SYN', 'ACK', 'RST', 'FIN', 'PSH', 'None']
}

# Convert to DataFrame
input_df = pd.DataFrame([input_data])

# Make prediction
prediction = model.predict(input_df)

# Get prediction probability
probability = model.predict_proba(input_df)[0]

# Output result
print("\n--- DDoS Detection Result ---")
print(f"Input Features: {input_data}")
print(f"Prediction: {'DDoS Attack (1)' if prediction[0] == 1 else 'Normal Traffic (0)'}")
print(f"Confidence: {probability[prediction[0]]*100:.2f}%")
print("-----------------------------")


--- DDoS Detection Result ---
Input Features: {'destination_port': '443', 'protocol': 'TCP', 'packet_count': 95, 'packet_size': 273, 'duration': 53, 'tcp_flags': 'ACK'}
Prediction: Normal Traffic (0)
Confidence: 100.00%
-----------------------------
